In [ ]:
# del session.logger
# del session

import multiprocessing
from train.training_session_v1 import TrainingSessionV1
from config import TrainingConfigV1
from config import SimpleConvConfig

data_partition = {
    "gwilliams2023": {
        "testing_subjects": [19, 20, 21],
        "testing_tasks": [0],
    },
    # "armeini2022": {
    #     "testing_subjects": [],
    #     "testing_tasks": [8, 9],
    # },
}

model_config = SimpleConvConfig(
    # Str to list of possible conditions
    mel_normalization=False,
    conditions={
        "study": [],
        "subject": [],
    },
    # Channels
    in_channels=208,
    out_channels=80,
    hidden_dim=384,
    dropout=0.2,
    initial_batch_norm=True,
    # Sensor layout settings
    layout_dim=2,
    layout_proj=True,
    layout_scaling="minmax",
    # Merger with spatial attn
    merger=False,
    merger_emb_type=None,
    merger_emb_dim=0,
    merger_channels=0,
    merger_dropout=0.0,  # Float
    merger_conditional=None,
    # Inital
    initial_linear=384,
    initial_depth=1,
    # Conditional layers
    conditional_layers=False,
    conditional_layers_dim=None,  # input or hidden_dim
    # Conv layer overall structure
    depth=6,
    kernel_size=3,
    growth=1.0,
    dilation_growth=2,
    dilation_period=5,
    glu=1,
    conv_dropout=0.2,
    dropout_input=0.2,
    batch_norm=True,
    half=True,
    cnn_pos_encoding=False,
    # Quantizer
    quantizer=False,
    num_codebooks=0,
    codebook_size=0,
    quantizer_commitment=0,
    quantizer_temp_init=0,
    quantizer_temp_min=0,
    quantizer_temp_decay=0,
    # Transformers Encoders
    transformer_input=None,
    transformer_encoder_emb=None,
    transformer_encoder_layers=0,
    transformer_encoder_heads=0,
    # Conformer encoder variant
    rnn_type="conformer",
    depthwise_conv_kernel_size=31,
    use_group_norm=True,
    convolution_first=False,
    # Transformer Decoders
    transformer_decoder_emb=None,
    transformer_decoder_layers=0,
    transformer_decoder_heads=0,
    transformer_decoder_dim=0,
)

config = TrainingConfigV1(
    brain_encoder_config=model_config,
    data_partition=data_partition,
    # Ada lora settings
    # Around 100k total batches an epoch for gwilliams
    adalora_init_r=12,
    adalora_target_r=4,
    adalora_tinit=2400,  # 5% total steps
    adalora_tfinal=25600,  # 50-80% total steps
    adalora_deltaT=1600,  # 1-5% total steps
    adalora_lora_alpha=32,
    adalora_lora_dropout=0.1,
    adalora_total_step=32000,
    # Pre-processing parameters
    # Brain
    new_freq=200,
    frequency_bands={"all": (0.5, 80)},
    max_random_shift=1.0,
    window_size=4,
    window_stride=1,
    brain_clipping=None,
    baseline_window=0.5,
    notch_filter=True,
    scaling="both",
    delay=0.15,
    # Hyperparameters
    learning_rate=3e-4,
    weight_decay=1e-4,
    epochs=40,
    batch_size=128,
    random_test_size=10,
    seed=42,
    mel_alignment_objectives={
        "clip_loss": 0.6,
        "mse_loss": 0.4,
        "commitment_loss": 0.0,
    },
    latent_alignment_objectives={
        "cosine_similarity": 0.4,
        "mse_loss": 0.4,
        "clip_loss": 0.2,
    },
    latent_alignment_layers=[-1],
)

config.learning_rate = 3e-3
config.brain_encoder_config.mel_normalization = True


session = TrainingSessionV1(
    config=config,
    studies={study: "audio" for study in data_partition.keys()},
    data_path="/home/ubuntu/storage/data",
    save_path="saves/phase2/objectives/CLIP_MSE_TEST2",
    clear_cache=False,
    cache_name="cache",
)

try:
    session.train(
        device="cuda",
        buffer_size=30,
        num_workers=(multiprocessing.cpu_count() - 2),
        max_cache_size=400,
        current_epoch=0,
    )
except KeyboardInterrupt as e:
    print("Exited")

# try:
#     session.pre_process_all_recordings(
#         buffer_size=30, num_workers=multiprocessing.cpu_count() - 2, max_cache_size=400
#     )
# except KeyboardInterrupt as e:
#     print("Exited")

Loading Gwilliams2023 with batch type audio
Data partitioned on studies ['gwilliams2023'].
Train: 135, Unseen Task: 12, Unseen Subject: 45, Unseen Both: 4.

SimpleConv initialized with 9295984 parameters, cond: ['study', 'subject']
Merger False, merger channels 0
ConvBlocks: 6, hidden_dim: 384, params 8858112
Found 36 target modules for AdaLora: ['k_proj', 'q_proj', 'v_proj', 'out_proj', 'fc1', 'fc2']
AdaLora model has 663984 parameters


2025-02-03 02:08:50,995	INFO worker.py:1841 -- Started a local Ray instance.
Training Epoch 1: 100%|██████████| 135/135 [13:22<00:00,  5.95s/it]


Epoch 1, Loss: 5.4294, Mel Loss: 3.4055
Clip Loss: 5.1532, MSE Loss: 0.7839, Commitment Loss: 0.0000
Perplexity: 0.0000, Accuracy: 0.0106, Top 5 Accuracy: 0.0491, Top 10 Accuracy: 0.0964
Final Layer Clip Loss: 6.0129, Final Layer MSE Loss: 1.5149, Final Layer Cosine Similarity Loss: 0.5384, Final Layer Total Loss: 2.0239
Test unseen_subject completed., Loss: 4.9074, Mel Loss: 3.2911
Clip Loss: 5.0142, MSE Loss: 0.7064, Commitment Loss: 0.0000
Perplexity: 0.0000, Accuracy: 0.0108, Top 5 Accuracy: 0.0483, Top 10 Accuracy: 0.1000
Final Layer Clip Loss: 5.2173, Final Layer MSE Loss: 1.1015, Final Layer Cosine Similarity Loss: 0.3306, Final Layer Total Loss: 1.6163
Test unseen_task completed., Loss: 5.1463, Mel Loss: 3.5063
Clip Loss: 5.3671, MSE Loss: 0.7151, Commitment Loss: 0.0000
Perplexity: 0.0000, Accuracy: 0.0077, Top 5 Accuracy: 0.0430, Top 10 Accuracy: 0.0884
Final Layer Clip Loss: 5.2861, Final Layer MSE Loss: 1.1225, Final Layer Cosine Similarity Loss: 0.3345, Final Layer Total L

Training Epoch 2: 100%|██████████| 135/135 [13:10<00:00,  5.85s/it]


Epoch 2, Loss: 4.9418, Mel Loss: 3.3460
Clip Loss: 5.1224, MSE Loss: 0.6815, Commitment Loss: 0.0000
Perplexity: 0.0000, Accuracy: 0.0089, Top 5 Accuracy: 0.0444, Top 10 Accuracy: 0.0889
Final Layer Clip Loss: 5.4021, Final Layer MSE Loss: 0.9984, Final Layer Cosine Similarity Loss: 0.2900, Final Layer Total Loss: 1.5958
Test unseen_subject completed., Loss: 6.4082, Mel Loss: 4.8373
Clip Loss: 7.7665, MSE Loss: 0.4434, Commitment Loss: 0.0000
Perplexity: 0.0000, Accuracy: 0.0098, Top 5 Accuracy: 0.0468, Top 10 Accuracy: 0.0990
Final Layer Clip Loss: 5.4808, Final Layer MSE Loss: 0.9203, Final Layer Cosine Similarity Loss: 0.2665, Final Layer Total Loss: 1.5709
Test unseen_task completed., Loss: 7.1231, Mel Loss: 5.5397
Clip Loss: 8.9317, MSE Loss: 0.4517, Commitment Loss: 0.0000
Perplexity: 0.0000, Accuracy: 0.0091, Top 5 Accuracy: 0.0413, Top 10 Accuracy: 0.0839
Final Layer Clip Loss: 5.4945, Final Layer MSE Loss: 0.9409, Final Layer Cosine Similarity Loss: 0.2705, Final Layer Total L

Training Epoch 3: 100%|██████████| 135/135 [13:10<00:00,  5.85s/it]


Epoch 3, Loss: 5.6301, Mel Loss: 4.1643
Clip Loss: 6.6187, MSE Loss: 0.4828, Commitment Loss: 0.0000
Perplexity: 0.0000, Accuracy: 0.0089, Top 5 Accuracy: 0.0438, Top 10 Accuracy: 0.0869
Final Layer Clip Loss: 5.1338, Final Layer MSE Loss: 0.8561, Final Layer Cosine Similarity Loss: 0.2414, Final Layer Total Loss: 1.4658
Test unseen_subject completed., Loss: 5.1888, Mel Loss: 3.7871
Clip Loss: 6.0766, MSE Loss: 0.3528, Commitment Loss: 0.0000
Perplexity: 0.0000, Accuracy: 0.0103, Top 5 Accuracy: 0.0497, Top 10 Accuracy: 0.0990
Final Layer Clip Loss: 4.9770, Final Layer MSE Loss: 0.7932, Final Layer Cosine Similarity Loss: 0.2225, Final Layer Total Loss: 1.4017
Test unseen_task completed., Loss: 5.6153, Mel Loss: 4.1793
Clip Loss: 6.7207, MSE Loss: 0.3673, Commitment Loss: 0.0000
Perplexity: 0.0000, Accuracy: 0.0086, Top 5 Accuracy: 0.0429, Top 10 Accuracy: 0.0863
Final Layer Clip Loss: 5.1019, Final Layer MSE Loss: 0.8125, Final Layer Cosine Similarity Loss: 0.2263, Final Layer Total L

Training Epoch 4:  15%|█▍        | 20/135 [02:30<14:26,  7.53s/it] 

Exited


In [ ]:
# import torch

# torch.cuda.get_device_capability()

In [ ]:
# for name, param in session.model.encoder.named_parameters():
#     if "lora_A" in name or "lora_B" in name or "lora_E" in name:
#         print(name)

In [2]:
# import time

# dataloader = session.get_dataloader(buffer_size=30, num_workers=24, max_cache_size=400)
# dataloader.start_fetching(session.dataset["train"], cache=True)

# # Process batches as they become available
# try:
#     batches, recs, start_time = 0, 0, time.time()
#     print(f"Total recordings: {len(session.dataset['train'])}")

#     while True:
#         batch = dataloader.get_recording()

#         if batch is None:
#             break

#         brain = batch.brain_segments["all"]
#         batches += brain.shape[0]
#         recs += 1

#         if recs % 10 == 0:
#             print(f'Processed {recs} recordings of {len(session.dataset["train"])}')

#     print(
#         f"Batch {batches} ({recs} recordings) processed in {time.time() - start_time:.2f}s"
#     )
#     print(
#         f"Average processing time per recording: {(time.time() - start_time) / recs:.2f}s"
#     )
#     print(
#         f"Average processing time per batch: {(time.time() - start_time) / batches:.2f}s"
#     )

# except KeyboardInterrupt:
#     print("Interrupted")
#     dataloader.stop()
# except Exception as e:
#     print("Error", e)
#     dataloader.stop()

2025-02-02 22:05:48,032	INFO worker.py:1841 -- Started a local Ray instance.


Total recordings: 135
Processed 10 recordings of 135
Processed 20 recordings of 135
Processed 30 recordings of 135
Processed 40 recordings of 135
Processed 50 recordings of 135
Processed 60 recordings of 135
Processed 70 recordings of 135
Processed 80 recordings of 135
Processed 90 recordings of 135
Processed 100 recordings of 135
Processed 110 recordings of 135
Processed 120 recordings of 135
Processed 130 recordings of 135
Batch 100895 (135 recordings) processed in 140.85s
Average processing time per recording: 1.04s
Average processing time per batch: 0.00s
